# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Cristóbal Ramos
- Nombre de alumno 2: Catalina Lizana 

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/CristobalRS/LABs_MDS7202-CrisTalina.git)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [1]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig

In [7]:
#!pip install -qqq gymnasium[box2d]

In [9]:
#pip install box2d-py

### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [11]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

**Respuesta:**


El ambiente Blackjack simula el juego del mismo nombre, donde un distribuidor va mostrando cartas al jugador con el fin que este obtenga una suma menor o igual a 21. Las cartas Jota, Queen y King tienen un valor de 10, la carta As vale 1 u 11 según conveniencia del jugador y el resto de cartas conserva su valor original.

**Estados**: Cada estado queda descrito por tres números:
* La suma de las cartas que tiene el jugador 
* La carta que muestra la ´mesa´ o el ´distribuidor´
* Un valor booleano (0 o 1) que indica si el jugador posee un As.

**Acciones**: Las acciones pueden ser 2 y son ejecutadas por el jugador: 
* 1 : Seguir jugando (Recibir carta)
* 0 : Parar (No recibir más cartas)

**Recompensas**: La recompensa se da al final de la partida y puede ser:
* +1: Gana el jugador
* -1: Pierde el Jugador
* 0: Empate 

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [13]:
#env = gym.make("Blackjack-v1", render_mode="human")
#env = gym.make("Blackjack-v1")
rew=[]
for episode in range(500):
    done = truncated = False
    state, info = env.reset()
    for timestep in range(100):
        action = env.action_space.sample()  #agent policy that uses the observation and info
        state, reward, done, truncated, info = env.step(action)
        if done or truncated:
            rew.append(reward)
            break

env.close()

In [15]:
promedio = np.mean(rew)
desviacion_estandar = np.std(rew, ddof=1)

print(f'Promedio: {promedio}')
print(f'Desviacion estandar: {desviacion_estandar}')

Promedio: -0.374
Desviacion estandar: 0.9031918461888615


**¿Cómo calificaría el performance de esta política?**

Notamos que no es una muy buena performance, ya que, como el objetivo es ganar esperariamos valores cercanos a 1 o por lo menos positivos.

**¿Cómo podría interpretar las recompensas obtenidas?**

Se interpreta como que tiende a ganar el distribuidor, lo cual es esperable considerando las condiciones del juego.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [17]:
from stable_baselines3 import DQN

# init agent
model = DQN("MlpPolicy", env, verbose=0)
# train the agent and display a progress bar
model.learn(total_timesteps=int(2e5), progress_bar=True)

Output()

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=500)
mean_reward, std_reward

(-0.058, 0.9626193432504876)

**¿Cómo es el performance de su agente?**

Es una performance más neutra ya que el promedio es más cercano a 0 y como std es aprox. $0.9$ significa que existe casi igual probabilidad que pierda o que gane el jugador 

**¿Es mejor o peor que el escenario baseline?**

Es mejor que el baseline.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [6]:
#env.reset()
def estado_accion(state):
    action, _states = model.predict(state, deterministic=True)
    action = action.item() 
    #state, reward, done, truncated, info = env.step(action)
    return action 
    

In [7]:
estado_accion((6,7,0))

1

In [8]:
estado_accion((19,3,1))

0

Notamos que los resultados son coherentes con el juego, pues, en el primer caso $6+7= 13<21$ por ende, sigue jugando y en el segundo se exceden los 21 por ende, se para el juego.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [2]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

El espacio de acciones es continuo, cuando continuous es False existen 4 acciones posibles, pero al ser True el espacio de acciones es el intervalo $[-1,1]^2$

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [3]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

**Respuesta:**
El objetivo del juego simulado es que la nave aterrice de forma segura en un espacio determinado.

**Estados**: Cada estado queda descrito por 8 datos
* La coordenada de la nave en x
* La coordenada de la nave en y
* La velocidad de la nave en x
* La velocidad de la nave en y
* El ángulo de inclinación de la nave
* La velocidad angular de la nave
* Un valor booleano que indica si la para derecha esta en el suelo 
* Un valor booleano que indica si la para izquierda esta en el suelo 

**Acciones**: El espacio de acciones esta compuesto por dos acciones, donde cada una de ellas pertenece al intervalo $[-1,1]$ 
* La primera coordenada controla el empuje vertical donde a mayor valor mayor es el empuje de la nave hacia arriba.
* La segunda coordenada controla el empuje lateral donde valores mayores indican un empuje hacia la derecha y valores menores indican un empuje hacia la izquierda.

**Recompensas**: La recompensa es acumulativa en los episodios y depende de :
* La distancia de la nave a la plataforma de aterrizaje. (Más cerca mejor puntaje)
* La velocidad de la nave (Más lento mejor puntaje)
* La inclinación de la nave (Menos inclinación mayor puntaje)
* Aumenta en 10 puntos por cada pata que esté en contacto con el suelo.
* Disminuye en 0,03 puntos cada fotograma en que se enciende un motor lateral.
* Disminuye en 0,3 puntos cada fotograma en que se enciende el motor principal.
* Aumenta 100 por aterrizar de forma segura
* Disminuye 100 por estrellarse 

Por último, un episodio se considera una solución si se obtienen al menos 200 puntos.

En Lunar Lander el espacio de acción tiene 2 dimensiones y cada una de ellas puede tomar infinitos valores (en $[-1,1]$), en cambio en Blackjack solo contamos con una dimensión que contempla 2 valores posibles. 

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [4]:
rew=[]
for episode in range(10):
    done = truncated = False
    state, info = env.reset()
    for timestep in range(150):
        action = env.action_space.sample()  
        state, reward, done, truncated, info = env.step(action)
        if done or truncated:
            #print(timestep)
            #print(reward)
            break
    rew.append(reward)
env.close()

In [5]:
promedio = np.mean(rew)
desviacion_estandar = np.std(rew, ddof=1)

print(f'Promedio: {promedio}')
print(f'Desviacion estandar: {desviacion_estandar}')

Promedio: -89.86266901401355
Desviacion estandar: 32.05705531071766


**¿Cómo calificaría el performance de esta política?**

Es una mala performance, al ser negativa y su std relativamente pequeño, significa que la nave no aterriza correctamente en practicamente ninguno de los episodios. Por otro lado, el valor promedio es muy lejano a 200.


#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [6]:
from stable_baselines3 import DDPG
from stable_baselines3 import TD3
from stable_baselines3 import SAC
from stable_baselines3 import PPO

env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) 

model = SAC("MlpPolicy", env, verbose=0)
model.learn(total_timesteps=int(1e4), progress_bar=True)

Output()

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [7]:
from stable_baselines3.common.evaluation import evaluate_policy

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
mean_reward, std_reward

(-32.1073605, 16.74173035370277)

**¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?**

Sigue siendo una mala performance por los mismos motivos explicados en la pregunta anterior, sin embargo, su desempeño es un poco mejor que el baseline. 




#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [14]:
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) 

model = SAC("MlpPolicy", env, learning_rate = 1e-4, batch_size = 512 , ent_coef="auto", verbose=1)
model.learn(total_timesteps = int(1.5e5) )
#progress_bar=True

Using cpu device

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 114      |
|    ep_rew_mean     | -148     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 51       |
|    time_elapsed    | 8        |
|    total_timesteps | 455      |
| train/             |          |
|    actor_loss      | -2.34    |
|    critic_loss     | 54.8     |
|    ent_coef        | 0.966    |
|    ent_coef_loss   | -0.111   |
|    learning_rate   | 0.0001   |
|    n_updates       | 354      |
---------------------------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 129      |
|    ep_rew_mean     | -181     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 45       |
|    time_elapsed    | 22       |
|    total_timesteps | 1033     |
| train/             |          |
|    actor_loss      | -1.67    |
|    critic_loss     | 29.5     |
|    ent_coef        | 0.914    |
|    ent_coef_loss   | -0.252   |
|    learning_rate   | 0.0001   |
|    n_updates       | 932      |
---------------------------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 137      |
|    ep_rew_mean     | -191     |
| time/              |          |
|    episodes        | 12       |
|    fps             | 44       |
|    time_elapsed    | 36       |
|    total_timesteps | 1641     |
| train/             |          |
|    actor_loss      | -1.97    |
|    critic_loss     | 28.9     |
|    ent_coef        | 0.864    |
|    ent_coef_loss   | -0.41    |
|    learning_rate   | 0.0001   |
|    n_updates       | 1540     |
---------------------------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 148      |
|    ep_rew_mean     | -211     |
| time/              |          |
|    episodes        | 16       |
|    fps             | 43       |
|    time_elapsed    | 54       |
|    total_timesteps | 2369     |
| train/             |          |
|    actor_loss      | -0.764   |
|    critic_loss     | 15.8     |
|    ent_coef        | 0.806    |
|    ent_coef_loss   | -0.584   |
|    learning_rate   | 0.0001   |
|    n_updates       | 2268     |
---------------------------------

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 620      |
|    ep_rew_mean     | 122      |
| time/              |          |
|    episodes        | 212      |
|    fps             | 36       |
|    time_elapsed    | 3860     |
|    total_timesteps | 140722   |
| train/             |          |
|    actor_loss      | -16.4    |
|    critic_loss     | 1.12     |
|    ent_coef        | 0.0428   |
|    ent_coef_loss   | -0.279   |
|    learning_rate   | 0.0001   |
|    n_updates       | 140621   |
---------------------------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 618      |
|    ep_rew_mean     | 124      |
| time/              |          |
|    episodes        | 216      |
|    fps             | 36       |
|    time_elapsed    | 3930     |
|    total_timesteps | 143363   |
| train/             |          |
|    actor_loss      | -17.3    |
|    critic_loss     | 4.21     |
|    ent_coef        | 0.0466   |
|    ent_coef_loss   | 0.616    |
|    learning_rate   | 0.0001   |
|    n_updates       | 143262   |
---------------------------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 610      |
|    ep_rew_mean     | 132      |
| time/              |          |
|    episodes        | 220      |
|    fps             | 36       |
|    time_elapsed    | 3982     |
|    total_timesteps | 145355   |
| train/             |          |
|    actor_loss      | -17.6    |
|    critic_loss     | 2.47     |
|    ent_coef        | 0.0481   |
|    ent_coef_loss   | 0.111    |
|    learning_rate   | 0.0001   |
|    n_updates       | 145254   |
---------------------------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 597      |
|    ep_rew_mean     | 138      |
| time/              |          |
|    episodes        | 224      |
|    fps             | 36       |
|    time_elapsed    | 4026     |
|    total_timesteps | 146986   |
| train/             |          |
|    actor_loss      | -17      |
|    critic_loss     | 24       |
|    ent_coef        | 0.0474   |
|    ent_coef_loss   | -0.249   |
|    learning_rate   | 0.0001   |
|    n_updates       | 146885   |
---------------------------------

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 586      |
|    ep_rew_mean     | 145      |
| time/              |          |
|    episodes        | 228      |
|    fps             | 36       |
|    time_elapsed    | 4088     |
|    total_timesteps | 149355   |
| train/             |          |
|    actor_loss      | -18.4    |
|    critic_loss     | 2.83     |
|    ent_coef        | 0.0452   |
|    ent_coef_loss   | 0.0258   |
|    learning_rate   | 0.0001   |
|    n_updates       | 149254   |
---------------------------------

Algunas observaciones:
* Recién en el episodio 116 tenemos valores de recompensa promedio postitivos
* A medida que aumentan los episodios se necesitan más pasos por episodios, en principio se usan alrededor de 100 y se termina con casi 600, sin embargo, en algunos episodios intermedios se usan más de 800 pasos.
* El tiempo de ejecución es proporcional al parámetro total_timesteps
* Al usar más timesteps, aumentamos el tamaño de los batch para optimizar el rendimiento
* No siempre que se usan los mismos parámetros, se obtienen los mismos resultados, sin embargo al aumentar el número de pasos de tiempo, y mantener los otros parámetros, los resultados varían menos pues la cantidad de episodios usados para entrenar es mayor y por ende, se obtienen métricas más concluyentes.
* El parámetro ent_coef="auto" controla el coeficiente de entopía automáticamente.

In [15]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
mean_reward, std_reward

(199.211389, 99.70792323773179)

In [16]:
#model.save("Lunar_lander_model") 
#model = SAC.load("Lunar_lander_model", env=env) 

Wrapping the env with a `Monitor` wrapper

Wrapping the env in a DummyVecEnv.

In [18]:
#export_gif(model)

![Texto alternativo](agent_performance.gif "Título opcional")

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Google AI API key:  ········
Enter your Tavily API key:  ········


### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [20]:
#%pip install --upgrade --quiet PyPDF2

In [3]:
import PyPDF2

doc_paths = ['HP1_Cap01.pdf', 'HP1_Cap02.pdf', 
             'HP1_Cap03.pdf', 'HP1_Cap04.pdf', 
             'HP1_Cap05.pdf', 'HP1_Cap06.pdf'] # Los 6 primeros capítulos de Harry Potter y la priedra filosofal

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader

# 0. Load 
doc1 = PyPDF2.PdfReader(doc_paths[0])
doc2 = PyPDF2.PdfReader(doc_paths[1])
doc3 = PyPDF2.PdfReader(doc_paths[2])
doc4 = PyPDF2.PdfReader(doc_paths[3])
doc5 = PyPDF2.PdfReader(doc_paths[4])
doc6 = PyPDF2.PdfReader(doc_paths[5])

In [7]:
#%pip install langchain_text_splitters
#%pip install --upgrade --quiet faiss-cpu langchain_community pypdf
#%pip install protobuf==3.20
#%pip install --upgrade --quiet  langchain-google-genai

In [8]:
from langchain.schema import Document
doc1_pages= [Document(page_content=page.extract_text()) for page in doc1.pages]
doc2_pages =[Document(page_content=page.extract_text()) for page in doc2.pages]
doc3_pages =[Document(page_content=page.extract_text()) for page in doc3.pages]
doc4_pages =[Document(page_content=page.extract_text()) for page in doc4.pages]
doc5_pages =[Document(page_content=page.extract_text()) for page in doc5.pages]
doc6_pages =[Document(page_content=page.extract_text()) for page in doc6.pages]

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

#1. Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(doc1_pages + doc2_pages + doc3_pages + doc4_pages + doc5_pages + doc6_pages) # dividir documentos en chunks

#2. Embed
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
vectorstore


In [14]:
#splits[0]

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [16]:
from langchain_core.runnables import RunnablePassthrough


retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3} )
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever_chain = retriever | format_docs # chain


In [20]:
question = "¿Quién es Harry Potter?" # pregunta
relevant_documents = retriever.invoke(question) 
print(relevant_documents)
print(retriever_chain.invoke(question))

[Document(metadata={}, page_content='nombre,\t¿verdad?\tToda\tesa\ttontería\tde\tQuien-usted-sabe…\tDurante\tonce\taños\tintenté\tpersuadir\ta\tla\tgente\npara\tque\tlo\tllamara\tpor\tsu\tverdadero\tnombre,\tVoldemort.\t—La\tprofesora\tMcGonagall\tse\techó\thacia\natrás\tcon\ttemor,\tpero\tDumbledore,\tocupado\ten\tdesenvolver\tdos\tcaramelos\tde\tlimón,\tpareció\tno\tdarse\ncuenta—.\tTodo\tse\tvolverá\tmuy\tconfuso\tsi\tseguimos\tdiciendo\t«Quien-usted-sabe».\tNunca\the\tencontrado\nningún\tmotivo\tpara\ttemer\tpronunciar\tel\tnombre\tde\tVoldemort.'), Document(metadata={}, page_content='estoy\tde\tacuerdo.\t¿Ya\tsabes\ten\tqué\tcasa\tvas\ta\testar?\n—No\t—dijo\tHarry,\tsintiéndose\tcada\tvez\tmás\ttonto.\n—Bueno,\tnadie\tlo\tsabrá\trealmente\thasta\tque\tlleguemos\tallí,\tpero\tyo\tsé\tque\tseré\tde\tSlytherin,\tporque\ntoda\tmi\tfamilia\tfue\tde\tallí.\t¿Te\timaginas\testar\ten\tHufflepuff?\tYo\tcreo\tque\tme\tiría,\t¿no\tte\tparece?\n—Mmm\t—contestó\tHarry,\tdeseando\tpoder\tdecir\

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [23]:
q1 = "¿Cuál es el nombre completo de Harry?"
q2 = "¿Quiénes son los personajes de apellido Weasley?"
q3 = "¿Qué hace Rubeus Hagrid en Hogwarts?"

# Respuestas correctas
RC1 = "Potter"
RC2 = "Ron, Fred y George"
RC3 = "Es el Guardián de las Llaves y Terrenos de Hogwarts"

a1= retriever_chain.invoke(q1)
a2= retriever_chain.invoke(q2)
a3= retriever_chain.invoke(q3)

print(f'PREGUNTA 1: {q1}')
print('-'*50)
print(a1)
print('='*140)

print(f'PREGUNTA 2: {q2}')
print('-'*50)
print(a2)
print('='*140)

print(f'PREGUNTA 3: {q3}')
print('-'*50)
print(a3)
print('='*140)

PREGUNTA 1: ¿Cuál es el nombre completo de Harry?
--------------------------------------------------
nombre,	¿verdad?	Toda	esa	tontería	de	Quien-usted-sabe…	Durante	once	años	intenté	persuadir	a	la	gente
para	que	lo	llamara	por	su	verdadero	nombre,	Voldemort.	—La	profesora	McGonagall	se	echó	hacia
atrás	con	temor,	pero	Dumbledore,	ocupado	en	desenvolver	dos	caramelos	de	limón,	pareció	no	darse
cuenta—.	Todo	se	volverá	muy	confuso	si	seguimos	diciendo	«Quien-usted-sabe».	Nunca	he	encontrado
ningún	motivo	para	temer	pronunciar	el	nombre	de	Voldemort.

decirle	que	había	oído	el	apellido	«Potter».	No,	no	se	atrevería.	En	lugar	de	eso,	dijo,	tratando	de
parecer	despreocupado:
—El	hijo	de	ellos…	debe	de	tener	la	edad	de	Dudley,	¿no?
—Eso	creo	—respondió	la	señora	Dursley	con	rigidez.
—¿Y	cómo	se	llamaba?	Howard,	¿no?
—Harry.	Un	nombre	vulgar	y	horrible,	si	quieres	mi	opinión.
—Oh,	sí	—dijo	el	señor	Dursley,	con	una	espantosa	sensación	de	abatimiento—.	Sí,	estoy	de
acuerdo.

—No	estoy	tratand

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [25]:
def hiper_para(tamaño_chuck, chuck_recuperados, tipo_busqueda, score_thre=1):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=tamaño_chuck, chunk_overlap=50) 
    splits = text_splitter.split_documents(doc1_pages + doc2_pages) 
    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") 
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding)
    retriever = vectorstore.as_retriever(search_type= tipo_busqueda, search_kwargs={"k": chuck_recuperados, "score_threshold": score_thre}) 
    retriever_chain = retriever | format_docs # chain

    
    a1= retriever_chain.invoke(q1)
    a2= retriever_chain.invoke(q2)
    a3= retriever_chain.invoke(q3)

    print(f'Respuestas con chuck_size = {tamaño_chuck}, chuck_recuperados = {chuck_recuperados} y tipo de busqueda = {tipo_busqueda}')
    print('='*140)
    
    print(f'PREGUNTA 1: {q1}')
    print('-'*50)
    print(a1)
    print('='*140)

    print(f'PREGUNTA 2: {q2}')
    print('-'*50)
    print(a2)
    print('='*140)
    
    print(f'PREGUNTA 3: {q3}')
    print('-'*50)
    print(a3)
    print('='*140)


In [27]:
hiper_para(100, 2, 'similarity')

Respuestas con chuck_size = 100, chuck_recuperados = 2 y tipo de busqueda = similarity
PREGUNTA 1: ¿Cuál es el nombre completo de Harry?
--------------------------------------------------
Escribirán	libros	sobre	Harry…	Todos	los	niños	del	mundo	conocerán	su	nombre.

siquiera	estaba	seguro	de	que	su	sobrino	se	llamara	Harry.	Nunca	había	visto	al	niño.	Podría	llamar
PREGUNTA 2: ¿Quiénes son los personajes de apellido Weasley?
--------------------------------------------------
que	la	señora	Dursley	fingía	que	no	tenía	hermana,	porque	su	hermana	y	su	marido,	un	completo	inúti

para	asimilarlo?
La	profesora	McGonagall	abrió	la	boca,	cambió	de	idea,	tragó	y	luego	dijo:
PREGUNTA 3: ¿Qué hace Rubeus Hagrid en Hogwarts?
--------------------------------------------------
Bueno,	déjalo	aquí,	Hagrid,	es	mejor	que	terminemos

—A	Hagrid,	le	confiaría	mi	vida	—dijo	Dumbledore.


**OBS:** En ninguna de las 3 llega a la información correcta

In [29]:
hiper_para(1000, 2, 'similarity') 

Respuestas con chuck_size = 1000, chuck_recuperados = 2 y tipo de busqueda = similarity
PREGUNTA 1: ¿Cuál es el nombre completo de Harry?
--------------------------------------------------
Por	otra	parte,	había	tenido	un	problema	terrible	cuando	lo	encontraron	en	el	techo	de	la	cocina	del
colegio.	El	grupo	de	Dudley	lo	perseguía	como	de	costumbre	cuando,	tanto	para	sorpresa	de	Harry	como
de	los	demás,	se	encontró	sentado	en	la	chimenea.	Los	Dursley	recibieron	una	carta	amenazadora	de	la
directora	del	colegio,	diciéndoles	que	Harry	andaba	trepando	por	los	techos	del	colegio.	Pero	lo	único
que	trataba	de	hacer	(como	le	gritó	a	tío	Vernon	a	través	de	la	puerta	cerrada	de	la	alacena)	fue	saltar	los
grandes	cubos	que	estaban	detrás	de	la	puerta	de	la	cocina.	Harry	suponía	que	el	viento	lo	había
levantado	en	medio	de	su	salto.
Pero	aquel	día	nada	iba	a	salir	mal.	Incluso	estaba	bien	pasar	el	día	con	Dudley	y	Piers	si	eso

ellos	fingían	que	ella	no	tenía	hermana.
—No	—respondió	en	tono	cortan

**OBS:** Solo en la primera pregunta llega a la información. 

In [31]:
hiper_para(200, 6, 'similarity')

Respuestas con chuck_size = 200, chuck_recuperados = 6 y tipo de busqueda = similarity
PREGUNTA 1: ¿Cuál es el nombre completo de Harry?
--------------------------------------------------
siquiera	estaba	seguro	de	que	su	sobrino	se	llamara	Harry.	Nunca	había	visto	al	niño.	Podría	llamarse

—Eso	creo	—respondió	la	señora	Dursley	con	rigidez.
—¿Y	cómo	se	llamaba?	Howard,	¿no?
—Harry.	Un	nombre	vulgar	y	horrible,	si	quieres	mi	opinión.

casa.
Cuando	era	más	pequeño,	Harry	soñaba	una	y	otra	vez	que	algún	pariente	desconocido	iba	a	buscarlo

Harry	tenía	un	rostro	delgado,	rodillas	huesudas,	pelo	negro	y	ojos	de	color	verde	brillante.	Llevaba

nombre,	¿verdad?	Toda	esa	tontería	de	Quien-usted-sabe…	Durante	once	años	intenté	persuadir	a	la	gente
para	que	lo	llamara	por	su	verdadero	nombre,	Voldemort.	—La	profesora	McGonagall	se	echó	hacia

comportando	como	un	estúpido.	Potter	no	era	un	apellido	tan	especial.	Estaba	seguro	de	que	había
muchísimas	personas	que	se	llamaban	Potter	y	que	tenían	un

**OBS:** Solo en la primera pregunta llega a la información correcta

In [33]:
hiper_para(200, 12, 'similarity')

Respuestas con chuck_size = 200, chuck_recuperados = 12 y tipo de busqueda = similarity
PREGUNTA 1: ¿Cuál es el nombre completo de Harry?
--------------------------------------------------
siquiera	estaba	seguro	de	que	su	sobrino	se	llamara	Harry.	Nunca	había	visto	al	niño.	Podría	llamarse

—Eso	creo	—respondió	la	señora	Dursley	con	rigidez.
—¿Y	cómo	se	llamaba?	Howard,	¿no?
—Harry.	Un	nombre	vulgar	y	horrible,	si	quieres	mi	opinión.

casa.
Cuando	era	más	pequeño,	Harry	soñaba	una	y	otra	vez	que	algún	pariente	desconocido	iba	a	buscarlo

Harry	tenía	un	rostro	delgado,	rodillas	huesudas,	pelo	negro	y	ojos	de	color	verde	brillante.	Llevaba

nombre,	¿verdad?	Toda	esa	tontería	de	Quien-usted-sabe…	Durante	once	años	intenté	persuadir	a	la	gente
para	que	lo	llamara	por	su	verdadero	nombre,	Voldemort.	—La	profesora	McGonagall	se	echó	hacia

comportando	como	un	estúpido.	Potter	no	era	un	apellido	tan	especial.	Estaba	seguro	de	que	había
muchísimas	personas	que	se	llamaban	Potter	y	que	tenían	u

In [34]:
hiper_para(200, 4, 'similarity')

Respuestas con chuck_size = 200, chuck_recuperados = 4 y tipo de busqueda = similarity
PREGUNTA 1: ¿Cuál es el nombre completo de Harry?
--------------------------------------------------
siquiera	estaba	seguro	de	que	su	sobrino	se	llamara	Harry.	Nunca	había	visto	al	niño.	Podría	llamarse

—Eso	creo	—respondió	la	señora	Dursley	con	rigidez.
—¿Y	cómo	se	llamaba?	Howard,	¿no?
—Harry.	Un	nombre	vulgar	y	horrible,	si	quieres	mi	opinión.

casa.
Cuando	era	más	pequeño,	Harry	soñaba	una	y	otra	vez	que	algún	pariente	desconocido	iba	a	buscarlo

Harry	tenía	un	rostro	delgado,	rodillas	huesudas,	pelo	negro	y	ojos	de	color	verde	brillante.	Llevaba
PREGUNTA 2: ¿Quiénes son los personajes de apellido Weasley?
--------------------------------------------------
El	señor	Dursley	era	el	director	de	una	empresa	llamada	Grunnings,	que	fabricaba	taladros.	Era	un
hombre	corpulento	y	rollizo,	casi	sin	cuello,	aunque	con	un	bigote	inmenso.	La	señora	Dursley	era

comportando	como	un	estúpido.	Potter	no	era	un

In [44]:
hiper_para(200, 4,'similarity_score_threshold', 0.3)

Respuestas con chuck_size = 200, chuck_recuperados = 4 y tipo de busqueda = similarity_score_threshold
PREGUNTA 1: ¿Cuál es el nombre completo de Harry?
--------------------------------------------------
siquiera	estaba	seguro	de	que	su	sobrino	se	llamara	Harry.	Nunca	había	visto	al	niño.	Podría	llamarse

—Eso	creo	—respondió	la	señora	Dursley	con	rigidez.
—¿Y	cómo	se	llamaba?	Howard,	¿no?
—Harry.	Un	nombre	vulgar	y	horrible,	si	quieres	mi	opinión.

casa.
Cuando	era	más	pequeño,	Harry	soñaba	una	y	otra	vez	que	algún	pariente	desconocido	iba	a	buscarlo

Harry	tenía	un	rostro	delgado,	rodillas	huesudas,	pelo	negro	y	ojos	de	color	verde	brillante.	Llevaba
PREGUNTA 2: ¿Quiénes son los personajes de apellido Weasley?
--------------------------------------------------
El	señor	Dursley	era	el	director	de	una	empresa	llamada	Grunnings,	que	fabricaba	taladros.	Era	un
hombre	corpulento	y	rollizo,	casi	sin	cuello,	aunque	con	un	bigote	inmenso.	La	señora	Dursley	era

comportando	como	un	estúpido.	

In [36]:
hiper_para(100, 1,'mmr')

Respuestas con chuck_size = 100, chuck_recuperados = 1 y tipo de busqueda = mmr
PREGUNTA 1: ¿Cuál es el nombre completo de Harry?
--------------------------------------------------
Escribirán	libros	sobre	Harry…	Todos	los	niños	del	mundo	conocerán	su	nombre.
PREGUNTA 2: ¿Quiénes son los personajes de apellido Weasley?
--------------------------------------------------
que	la	señora	Dursley	fingía	que	no	tenía	hermana,	porque	su	hermana	y	su	marido,	un	completo	inúti
PREGUNTA 3: ¿Qué hace Rubeus Hagrid en Hogwarts?
--------------------------------------------------
Bueno,	déjalo	aquí,	Hagrid,	es	mejor	que	terminemos


**OBS:** Esta última hacía que se cayera el kernel, por eso la dejamos con tan poco tamaño

Si los chunks son muy pequeños, aumenta la probabilidad de que la información encontrada no sea de utilidad para responder, pues pueden haber similitudes entre la pregunta y la información seleccionada, pero no estar relacionadas ya que no cuenta con contexto. 
Por otro lado, si los chunks son muy grandes, selecciona mucho texto y entre el cual hay mucha información que no es de utilidad para responder. 

En general, con pocos chuck_recuperados, se llega muy pocas veces a obtener información para responder correctamente. Fijando este parámetro, por ejemplo en 5, le damos 5 oportunidades al modelo de que la información seleccionada sea de utilidad. Por lo que, con más chunks, aumenta la probabilidad de encontrar información de valor, siempre y cuando su tamaño sea suficiente como para que la información seleccionada también sea de utilidad. 
)Variando e
El tipo de búsque resultan distintas respuestas, esto pues, por ejemplo seleccionando similarity_score_threshold permitimos que la similutud buscada en la infrmación no sea siempre del 100%, permintiendo respuestas más variadas. Esta mejora o empeora las erespuestas segun cada caso. ?)

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [47]:
from langchain_community.tools.tavily_search import TavilySearchResults
search_tavily = TavilySearchResults(max_results = 1) # inicializamos tool


#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [50]:
#pip install wikipedia

In [52]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper( top_k_results=1, doc_content_chars_max=100)
search_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
tools = [search_tavily, search_wiki] # guardamos las tools en una lista

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [55]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import hub
from langchain_core.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)

react_prompt =  '''
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. Print the answer in spanish.

Begin!

Question: {input}
Thought:{agent_scratchpad}

'''

prompt = PromptTemplate.from_template(react_prompt)
print(prompt.template)


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. Print the answer in spanish.

Begin!

Question: {input}
Thought:{agent_scratchpad}




In [57]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents import create_tool_calling_agent

agent = create_react_agent(llm, tools, prompt) # inicializamos el agente ReAct
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.', 'tool_names': 'tavily_search_results_json, wikipedia'}, template='\nAnswer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the 

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [62]:
response = agent_executor.invoke({"input": "qué equipo ganó el mundial de LoL 2024?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: The question asks about the winner of the 2024 League of Legends World Championship.  This is a current event, and the tournament hasn't happened yet, so I need to check if any information is available, but I expect to find no results.

Action: tavily_search_results_json
Action Input: "League of Legends World Championship 2024 winner"
[{'url': 'https://insider-gaming.com/lol-worlds-2024-winner/', 'content': 'T1 Wins League of Legends World Championship 2024 - Insider Gaming G Gaming World Tour Gaming World Tour T1 had a massive lead on towers and in gold, and it proved to be too much for BLG, with T1 winning the game before a screaming crowd and a kill record of 14 – 9 over BLG. For more Insider Gaming esports, check out the news that an OpTic-FaZe combination won a Black Ops 6 tournament T1’s Faker has secured a 500-kill record at the League of Legends World Championship, which has been accrued over ten years of competitive history. Lea

¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

La tool de tavily se debería ocupar para preguntas más específica, y en partiuclar, cuando se quiere de información más actualizada. Por otro lado, Wikipedia se podría utilizar para temas más generales, que no requieran información puntual, específica, o qeu tenga qeu estar actualizada para su validez. Un ejemplo de esto son las respuestas frente a la pregunta del ganador del mundial de LOL, según wikipedia aun no había ganador, según tavily (qeu buscó información más actualizada) si había un ganador. 

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [64]:
from langchain.tools import tool
@tool
def chain_tool(input: str):
    """Responde en base a pdf"""
    return retriever_chain.invoke(input)

@tool
def agent_tool(input: str):
    """Responde usando wikipedia o tavily"""
    return agent_executor.invoke(input)

tools = [chain_tool, agent_tool]

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)


template = """
Respondes preguntas lo mejor que puedas
Pregunta: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='\nRespondes preguntas lo mejor que puedas\nPregunta: {input}\n{agent_scratchpad}\n')

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [67]:
agent = create_tool_calling_agent(llm, tools, prompt)
Supervisor = AgentExecutor(agent=agent, tools=tools, verbose=True)
Supervisor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='\nRespondes preguntas lo mejor que puedas\nPregunta: {input}\n{agent_scratchpad}\n')
| RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000011E70D523C0>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'chain_tool', 'description': 'Responde en base a pdf', 'parameters': {'properties': {'input': {'type': 'string'}}, 'required': ['input'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'agent_tool', 'description': 'Responde usando wiki

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [74]:
q1 = "¿Cuál es el nombre completo de Harry?"
q2 = "¿Quiénes son los personajes de apellido Weasley?"
q3 = "¿Qué hace Rubeus Hagrid en Hogwarts?"

Supervisor.invoke({"input": q2})



> Entering new AgentExecutor chain...

Invoking: `chain_tool` with `{'input': '¿Quiénes son los personajes de apellido Weasley?'}`


nombre,	¿verdad?	Toda	esa	tontería	de	Quien-usted-sabe…	Durante	once	años	intenté	persuadir	a	la	gente
para	que	lo	llamara	por	su	verdadero	nombre,	Voldemort.	—La	profesora	McGonagall	se	echó	hacia
atrás	con	temor,	pero	Dumbledore,	ocupado	en	desenvolver	dos	caramelos	de	limón,	pareció	no	darse
cuenta—.	Todo	se	volverá	muy	confuso	si	seguimos	diciendo	«Quien-usted-sabe».	Nunca	he	encontrado
ningún	motivo	para	temer	pronunciar	el	nombre	de	Voldemort.

Ron	dejó	escapar	una	débil	tos,	que	podía	estar	ocultando	una	risita.	Draco	(dragón)	Malfoy	lo	miró.
—Te	parece	que	mi	nombre	es	divertido,	¿no?	No	necesito	preguntarte	quién	eres.	Mi	padre	me	dijo
que	todos	los	Weasley	son	pelirrojos,	con	pecas	y	más	hijos	que	los	que	pueden	mantener.
Se	volvió	hacia	Harry.

Scabbers
,	la	rata,	colgaba	del	dedo	de	Goyle,	con	los	agudos	dientes	clavados	profundamente	en	sus

{'input': '¿Quiénes son los personajes de apellido Weasley?',
 'output': 'El texto proporcionado menciona a los Weasley, pero no lista a todos los personajes con ese apellido.  Para obtener una lista completa de los personajes Weasley, se necesita más información o una fuente diferente.\n'}

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.


La principal diferencia es que este enfoque el agente elige que tool usar, en cambio en router se le dan una serie de instrucciones para que determine el agente. La ventaja de este enfoque es que es más simple de implementar y de entender y su desventaja es que no podemos estar seguros que la tool que usa entrega la mejor respuesta a la pregunta.

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [ ]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )